In [140]:
import pymysql
connection = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'dan20111965',
    db = 'dot',
    charset = 'utf8'
)

In [141]:
cursor = connection.cursor()
cursor.execute("show tables")

1

In [142]:
cursor.fetchall()

(('my',),)

## Задание №2

In [143]:
import csv

In [144]:
f = lambda words: '"' + words + '"'
temp = 0
with open("data.csv") as File:
    for row in File:
        words_arr = row.split(',')
        words_arr = list(map(f, words_arr[0: min(20, len(words_arr))]))
        words_arr[-1] = words_arr[-1].replace("\n", "")
        
        columns = ["id_f"] + [f"a{j + 1}" for j in range(len(words_arr))]
        
        quer = f"INSERT INTO `my` ({','.join(columns)}) VALUES ({temp}, {','.join(words_arr)});"
        temp += 1
        cursor.execute(quer)

In [145]:
connection.commit()

## Задание №3

In [146]:
import atoti as tt
session = tt.Session()

Welcome to atoti 0.6.5!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [150]:
import atoti_sql

In [151]:
my_table = session.read_sql(
    query = 'SELECT * FROM my;',
    url = "jdbc:mysql://localhost:3306/dot?user=root&password=dan20111965",
    table_name = 'Таблица из CSV',
    keys = ['id_f'],
    driver = atoti_sql.drivers.MYSQL
)

In [153]:
my_table.head()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20
id_f,,,,,,,,,,,,,,,,,,,,
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,chutney,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
307,frozen vegetables,spaghetti,yams,mineral water,french wine,green grapes,rice,chocolate,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,turkey,avocado,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [154]:
cube = session.create_cube(my_table)

In [264]:
lst_col = list()
for i in range(1, 21):
    lst_col.append(f"a{i}")
print(lst_col)

['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20']


In [332]:
task3_lst = list()
for i in range(len(lst_col)):
    cursor.execute(f"SELECT COUNT({lst_col[i]}) FROM `my` WHERE {lst_col[i]} IS NOT NULL;")
    task3_lst.append(f"Столбец {lst_col[i]}: {int(''.join(map(str,cursor.fetchone())))}")
task3_lst

['Столбец a1: 7501',
 'Столбец a2: 5747',
 'Столбец a3: 4389',
 'Столбец a4: 3345',
 'Столбец a5: 2529',
 'Столбец a6: 1864',
 'Столбец a7: 1369',
 'Столбец a8: 981',
 'Столбец a9: 654',
 'Столбец a10: 395',
 'Столбец a11: 256',
 'Столбец a12: 154',
 'Столбец a13: 87',
 'Столбец a14: 47',
 'Столбец a15: 25',
 'Столбец a16: 8',
 'Столбец a17: 4',
 'Столбец a18: 4',
 'Столбец a19: 3',
 'Столбец a20: 1']

## Задание №4

Скриншот созданного кластера закрепил в .docx файле.

## Задание №5

In [294]:
import redis
import rediscluster
import uuid

In [295]:
redcon = rediscluster.RedisCluster(host = 'localhost', port = 7001)
redcon.flushdb()
print(redcon)

RedisCluster<localhost:7001>


In [296]:
cursor.execute("SELECT * FROM `my`;")
num5_arr = cursor.fetchall()

In [297]:
print(num5_arr[0])

(0, 'shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil')


In [298]:
for words in num5_arr[0:]:
    arr_uuid = uuid.uuid4().hex
    lst_witout_null = [i for i in words if i is not None]
    for j in lst_witout_null:
        redcon.rpush(arr_uuid, j)

In [299]:
print(redcon.keys()[0: 10])

[b'8092e3b01872419f92b3e1c08192707b', b'a08c3cc9eb4f4f098924b51c81f3b17d', b'9b27e7f5bbc04cf58313c3039ee138b4', b'026f6d5d3ebd4f2ba7941535042c3eb9', b'bb49014be549406099f500607990dc5e', b'ee02c8d9e62241e1981fe55adf376f0a', b'c820f131d4154728be86515dcb466cf7', b'734d2f82c0e44109b6d3d0625d284ed3', b'4246804c7f7347e5992633909a56df26', b'a6645b4f605d42908bc31d010d888ead']


In [300]:
redcon.lrange('8092e3b01872419f92b3e1c08192707b', 0, -1)

[b'2466', b'chocolate', b'fresh bread', b'honey']

## Задание №6

In [301]:
num6_dict = dict()
# Считываю все ключи из Redis.
arr_keys = redcon.keys()
# В цикле сначала беру по ключу Reids'а список, который он хранит.
# Далее устанавливаю значение ключа в словаре, оно идентично самому первому значение из списка Redis (так называемый id_f в моем случае).
# Значением по данному ключу будет длина оставшегося списка temp, в который ранее я сохранил весь список по ключу i из Redis.
for i in arr_keys:
    temp = redcon.lrange(i, 0, -1)
    num6_dict[temp[0].decode("utf-8")] = len(temp[1:])

In [302]:
for j in list(num6_dict.items())[0: 5]:
    print(j)

('2466', 3)
('1108', 2)
('3100', 7)
('3406', 3)
('6763', 3)


In [303]:
hashTabName = "Task6"
for key, val in zip(num6_dict.keys(), num6_dict.values()):
    redcon.hset(hashTabName, mapping = {key: val})

In [318]:
arr_k = redcon.hkeys(hashTabName)[0: 15]
for i in range(len(arr_k)):
    print(arr_k[i], redcon.hget(hashTabName, arr_k[i]))

b'6381' b'2'
b'2655' b'1'
b'5567' b'2'
b'2562' b'2'
b'7494' b'12'
b'6874' b'6'
b'7315' b'4'
b'646' b'5'
b'3608' b'7'
b'4981' b'6'
b'5139' b'3'
b'1689' b'3'
b'4089' b'7'
b'337' b'3'
b'3766' b'4'


## Задание №7

In [320]:
import random

In [328]:
def rand_lst_word(rand_arr_key_word):
    # Забираю все ключи из Redis. 
    arr_keys = redcon.keys()
    # Рандомно выбирается число ключей равное аргументу функции.
    rand_keys = random.choices(arr_keys, k = rand_arr_key_word)
    # Отбор ключей, которых нет в hash-таблице.
    while hashTabName in [key.decode("UTF-8") for key in rand_keys]:
        rand_keys = random.choices(arr_keys, k = rand_arr_key_word)
    # Выбираю рандомное значение (слово) при каждой новой итерации цикла, ключи берутся из списка, в котором нет ключей из hash-таблицы.
    # В lrange поиск значения начинаю с 1-го элемента, а не 0-го, т.к. 0-ой - номер строки.
    arr_val = [random.choice(redcon.lrange(j, 1, -1)).decode("UTF-8") for j in rand_keys]
    # Конкатенация получившихся слов через символ пробела.
    return ' '.join(arr_val)

for i in range(10):
    print("Ключ ", i, " ... ", "[", rand_lst_word(random.randint(1, 20)), "]")
    

Ключ  0  ...  [ eggs olive oil mineral water ]
Ключ  1  ...  [ avocado milk spaghetti butter salmon eggs burgers milk mineral water soda meatballs spaghetti pet food green tea frozen smoothie french fries mushroom cream sauce french fries butter green beans ]
Ключ  2  ...  [ eggs escalope blueberries pancakes burgers avocado ]
Ключ  3  ...  [ red wine pasta ground beef hot dogs frozen smoothie french fries eggplant mushroom cream sauce mineral water salad chocolate yogurt cake milk vegetables mix mineral water honey escalope spaghetti pancakes ]
Ключ  4  ...  [ french fries green tea red wine frozen vegetables cookies pasta milk ]
Ключ  5  ...  [ cookies milk eggs eggs frozen smoothie turkey herb & pepper olive oil spaghetti green tea avocado spaghetti eggs spaghetti french fries burgers turkey cooking oil whole weat flour french fries ]
Ключ  6  ...  [ frozen smoothie burgers mineral water pancakes brownies oil pancakes ]
Ключ  7  ...  [ mayonnaise energy bar cider oil french fries fl

## Задание №8

Скриншот созданного кластера в MongoDB закрепил в .docx файле.

## Задание №9

In [395]:
import pymongo as pm

In [438]:
# Подключение к серверу через один из маршрутизаторов
client = pm.MongoClient('localhost', 40000)

# Подключение к базе данных
db = client['mongoTask']

# Получение доступа к коллекции
collection = db['mycollect']

In [440]:
collection.drop()

In [441]:
for i in range(1000):
    # Т.к. в 10-ом номере необходимо брать name1 и name2, значит, при вызове фукции из 7-го номера
    # кол-во слов должно быть от 3-х минимум.
    new_arr_words = rand_lst_word(random.randint(3, 20))
    arr_num9 = new_arr_words.split(' ')
    new_keys = [f"name{j}" for j in range(len(arr_num9))]
    collection.insert_one({key: val for key, val in zip(new_keys, arr_num9)})

In [442]:
mongoData = collection.find()
for i in range(5):
    print(mongoData[i])

{'_id': ObjectId('62a378b6fccc6037c546d9a3'), 'name0': 'whole', 'name1': 'wheat', 'name2': 'rice', 'name3': 'chocolate', 'name4': 'french', 'name5': 'fries', 'name6': 'almonds', 'name7': 'escalope', 'name8': 'burgers', 'name9': 'grated', 'name10': 'cheese', 'name11': 'tomato', 'name12': 'juice', 'name13': 'mineral', 'name14': 'water', 'name15': 'strawberries', 'name16': 'energy', 'name17': 'drink', 'name18': 'mayonnaise', 'name19': 'cookies', 'name20': 'spaghetti', 'name21': 'olive', 'name22': 'oil', 'name23': 'white', 'name24': 'wine', 'name25': 'green', 'name26': 'tea'}
{'_id': ObjectId('62a378b7fccc6037c546d9a4'), 'name0': 'low', 'name1': 'fat', 'name2': 'yogurt', 'name3': 'meatballs', 'name4': 'low', 'name5': 'fat', 'name6': 'yogurt', 'name7': 'low', 'name8': 'fat', 'name9': 'yogurt', 'name10': 'grated', 'name11': 'cheese', 'name12': 'frozen', 'name13': 'smoothie', 'name14': 'olive', 'name15': 'oil', 'name16': 'whole', 'name17': 'wheat', 'name18': 'rice', 'name19': 'cookies', 'name

## Задание №10

In [443]:
need_record = list()
for val in mongoData:
    # Если функция find не нашла никаких записей, возвращает -1, поэтому условие !=. 
    if(val['name1'].find('a') != -1) and (val['name2'].find('m') != -1):
        need_record.append(val)  

In [447]:
for i in range(10):
    print(need_record[i], '\n')

{'_id': ObjectId('62a378b7fccc6037c546d9a9'), 'name0': 'yogurt', 'name1': 'cake', 'name2': 'salmon', 'name3': 'nonfat', 'name4': 'milk', 'name5': 'chocolate', 'name6': 'mineral', 'name7': 'water', 'name8': 'turkey', 'name9': 'mineral', 'name10': 'water'} 

{'_id': ObjectId('62a378b9fccc6037c546d9c2'), 'name0': 'frozen', 'name1': 'vegetables', 'name2': 'milk', 'name3': 'chocolate', 'name4': 'mineral', 'name5': 'water', 'name6': 'spinach', 'name7': 'shrimp', 'name8': 'nonfat', 'name9': 'milk', 'name10': 'pickles', 'name11': 'brownies', 'name12': 'blueberries'} 

{'_id': ObjectId('62a378bcfccc6037c546d9db'), 'name0': 'protein', 'name1': 'bar', 'name2': 'tomatoes', 'name3': 'french', 'name4': 'wine', 'name5': 'fresh', 'name6': 'bread', 'name7': 'spaghetti', 'name8': 'eggs', 'name9': 'eggs', 'name10': 'avocado', 'name11': 'cake', 'name12': 'burgers', 'name13': 'salmon', 'name14': 'oil', 'name15': 'soup', 'name16': 'green', 'name17': 'tea', 'name18': 'soup', 'name19': 'light', 'name20': 'may